# SECOND EXPERIMENTATION - FIRST ASSUMPTION, WITHOUT LABELING CORRECTION

## Training a sentiment analysis classifier based on supervised machine learning algorithms

In [1]:
import string

import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import ConfusionMatrixDisplay, precision_score, recall_score, f1_score

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
stop_words = set(stopwords.words('spanish'))

In [4]:
def tokenizer(text):
    tt = TweetTokenizer()
    return tt.tokenize(text)

### Loading labeled tweets

In [5]:
# Dataset loaded from: https://docs.google.com/spreadsheets/d/11_E2ngsEOyTQwbwVVRHY5urzFF95BQCV/edit#gid=1788161364
tweets_df = pd.read_csv('./data/tweets_labeled.csv', sep = ',')

In [6]:
tweets_df.shape

(296, 2)

In [7]:
tweets_df.head()

,full_text,sentiment
0,@Nata_Salud @Sandrag69 @AndresMejiaV ¡Hola Natalia! Te invitamos a descubrir tu #MatchPresidencial aquí: https://t.co/0E1tZKypTK,neutral
1,@supershadai @Registraduria Quien o que institución en Colombia atiende los reclamos al fraude electoral?\r\nPorque no suspender a al registrador que ya la defeco en las elecciones de senado y camara.\r\nHay una desconfianza general en cuanto a las presidenciales.\r\nEstán provocando una respuesta violenta.,negative
2,@BOLIBAR2 @AndresPastrana_ @santiagoangelp Un poco tarde con las elecciones encima… mal para Colombia,negative
3,"No encontraron otra alternativa que llenar de miedo a Colombia, utilizan sus paramilitares para ganar elecciones. Es ahora o nunca @petrogustavo",positive
4,"@BOLIBAR2 @CNE_COLOMBIA @AndresPastrana_ Aquí no va a pasar nada, y petro de va a robar las elecciones presidenciales y el país",negative


In [8]:
df2=tweets_df.replace("neutral","positive/neutral")

In [9]:
df2=df2.replace("positive","positive/neutral")

In [10]:
df2.head()

,full_text,sentiment
0,@Nata_Salud @Sandrag69 @AndresMejiaV ¡Hola Natalia! Te invitamos a descubrir tu #MatchPresidencial aquí: https://t.co/0E1tZKypTK,positive/neutral
1,@supershadai @Registraduria Quien o que institución en Colombia atiende los reclamos al fraude electoral?\r\nPorque no suspender a al registrador que ya la defeco en las elecciones de senado y camara.\r\nHay una desconfianza general en cuanto a las presidenciales.\r\nEstán provocando una respuesta violenta.,negative
2,@BOLIBAR2 @AndresPastrana_ @santiagoangelp Un poco tarde con las elecciones encima… mal para Colombia,negative
3,"No encontraron otra alternativa que llenar de miedo a Colombia, utilizan sus paramilitares para ganar elecciones. Es ahora o nunca @petrogustavo",positive/neutral
4,"@BOLIBAR2 @CNE_COLOMBIA @AndresPastrana_ Aquí no va a pasar nada, y petro de va a robar las elecciones presidenciales y el país",negative


In [11]:
#df2.to_csv('tweets - Modificados según resultados.csv',index=False)

In [12]:
#Manually change the tweets that were false positive and false negative
df2 = pd.read_csv('./data/tweets - Modificados según resultados.csv', sep = ',')

In [13]:
df2['sentiment'].value_counts(dropna = False, normalize = True)

negative            0.692568
positive/neutral    0.307432
Name: sentiment, dtype: float64

### Leaving out unlabeled texts, this data is not useful for training or validating a supervised model

In [14]:
# Removing  unlabeled tweets
tweets_labeled_df = df2.loc[tweets_df['sentiment'].notnull()]

In [15]:
tweets_labeled_df.shape

(296, 2)

In [16]:
tweets_unlabeled_df = df2.loc[df2['sentiment'].isnull()]

In [17]:
tweets_unlabeled_df.shape

(0, 2)

In [18]:
# Scenario 3: Working with 2 classes
tweets_labeled_df['sentiment'] = tweets_labeled_df['sentiment']

### Splitting train and test datasets

In [19]:
X_train, X_test, y_train, y_test = train_test_split(tweets_labeled_df['full_text'], tweets_labeled_df['sentiment'], test_size = 0.2, stratify = tweets_labeled_df['sentiment'], random_state = 1)


In [20]:
X_train.shape

(236,)

In [21]:
pd.Series(y_train).value_counts(normalize = True)

negative            0.690678
positive/neutral    0.309322
Name: sentiment, dtype: float64

In [22]:
X_test.shape

(60,)

In [23]:
pd.Series(y_test).value_counts(normalize = True)

negative            0.7
positive/neutral    0.3
Name: sentiment, dtype: float64

### Vectorizing texts

<table>
    <tbody>
        <tr>
            <td>
                <h4>Bag of Words</h4>
                <img src="imgs/bow.png" style="width: 500px;">
            </td>
            <td>
                <h4>TF-IDF</h4>
                <img src="imgs/tf-idf.png" style="width: 500px;">
            </td>
        </tr>
    </tbody>
</table>

In [24]:
bow = CountVectorizer(tokenizer = tokenizer, stop_words = stop_words)

In [25]:
tfidf = TfidfVectorizer(tokenizer = tokenizer, stop_words = stop_words)

In [26]:
X_bow = bow.fit_transform(X_train)

In [27]:
X_tfidf = tfidf.fit_transform(X_train)

### Training and evaluating a model using BOW

In [28]:
model = RandomForestClassifier()

In [29]:
model.fit(X_bow, y_train)

RandomForestClassifier()

In [30]:
y_train_bow_predict = model.predict(X_bow)
y_test_bow_predict = model.predict(bow.transform(X_test))

In [31]:
ConfusionMatrixDisplay.from_predictions(y_train, y_train_bow_predict)

AttributeError: type object 'ConfusionMatrixDisplay' has no attribute 'from_predictions'

In [32]:
ConfusionMatrixDisplay.from_predictions(y_test, y_test_bow_predict)

AttributeError: type object 'ConfusionMatrixDisplay' has no attribute 'from_predictions'

In [33]:
# Metrics calculation for more than two classes
print('Precision:', precision_score(y_test, y_test_bow_predict, average = None))
print('Recall:', recall_score(y_test, y_test_bow_predict, average = None))
print('F1:', f1_score(y_test, y_test_bow_predict, average = None))

Precision: [0.74074074 0.66666667]
Recall: [0.95238095 0.22222222]
F1: [0.83333333 0.33333333]


### Training and evaluating a model using TF-IDF

In [34]:
model = RandomForestClassifier()

In [35]:
model.fit(X_tfidf, y_train)

RandomForestClassifier()

In [36]:
y_train_tfidf_predict = model.predict(X_tfidf)
y_test_tfidf_predict = model.predict(bow.transform(X_test))

In [37]:
ConfusionMatrixDisplay.from_predictions(y_train, y_train_tfidf_predict)

AttributeError: type object 'ConfusionMatrixDisplay' has no attribute 'from_predictions'

In [38]:
ConfusionMatrixDisplay.from_predictions(y_test, y_test_tfidf_predict)

AttributeError: type object 'ConfusionMatrixDisplay' has no attribute 'from_predictions'

In [39]:
# Metrics calculation for more than two classes
print('Precision:', precision_score(y_test, y_test_tfidf_predict, average = None))
print('Recall:', recall_score(y_test, y_test_tfidf_predict, average = None))
print('F1:', f1_score(y_test, y_test_tfidf_predict, average = None))

Precision: [0.76595745 0.53846154]
Recall: [0.85714286 0.38888889]
F1: [0.80898876 0.4516129 ]


### How interpret the results?

### Analyzing errors Bag of Words

In [40]:
error_df1 = pd.concat(
    [ pd.concat([X_test, y_test ], axis = 1).reset_index(),
    pd.Series(y_test_bow_predict) ]
, axis = 1).rename(columns = { 'sentiment': 'actual', 0: 'predicted' })

error_df1.drop('index', inplace = True, axis = 1)

In [41]:
error_df1.shape

(60, 3)

In [42]:
error_df1.loc[error_df1['actual'] != error_df1['predicted']].head(20)

,full_text,actual,predicted
8,"@tinagus2000 Evidentemente la persecución judicial, la aceleración de cosas, las trabas del @cnegobec para la inscripción , el viaje de la fiscal a Colombia y el papel de otras instituciones influyeron en la victoria de Lasso. Al otro candidato lo desgastaron, mucha antes de las elecciones.",positive/neutral,negative
16,"Sin importar la orilla política en la que se esté, es preciso reconocer que Álvaro Uribe ha sido el que les dio forma a los últimos 20 años de la política colombiana. \r\n\r\n#ContenidoPremiumEE #Elecciones2022 \r\nhttps://t.co/tVxPTMLkC2",positive/neutral,negative
18,Petro promete reanudar relaciones diplomáticas con Maduro si gana elecciones en Colombia\r\n#TalCual #ClaroyRaspao #AmigosDeNuevo \r\nhttps://t.co/Kpm7VZydwZ https://t.co/xyy0uPwwzZ,negative,positive/neutral
20,Eso sabe ;)\r\nAdelante #PetroPresidente2022 \r\nhttps://t.co/xscQ26b62m https://t.co/NIJNT0Sms7,positive/neutral,negative
23,Me salió esto 😅\r\n\r\nhttps://t.co/TJokUajStC,positive/neutral,negative
24,"@condeza6 @Gonzalo00333993 @Pura_Miel @Registraduria @CNE_COLOMBIA @moecolombia @PGN_COL Coño que porquería de país , acaso la procuradora se revolcó con el registrador y le tomo fotos y videos que la tiene chantajeada o que , que carajos es lo que esa vieja está esperando que se forme un mierdero el día de las elecciones?",positive/neutral,negative
31,Estos son los tarjetones con los que Colombia ha votado a la presidencia en 28 años:\r\n\r\nhttps://t.co/7jcyCwQuni,positive/neutral,negative
32,#Política #Colombia | Uno de los vehículos portaba en su parabrisas un distintivo que lo identificaba como parte de la Alcaldía de Medellín.\r\nhttps://t.co/LDYchqsr08,positive/neutral,negative
33,Procuraduría alerta que 2.925 jurados de votación están inhabilitados para elecciones Colombia via @ElColombiano https://t.co/lCdwAtuKoM,negative,positive/neutral
37,"@correawilly @CNE_COLOMBIA Leído los documentos provenientes de cada circunscripción territorial y de la internacional, se declaró el resultado de la Cámara por circunscripción internacional y se recibieron más de 1000 solicitudes y reclamaciones respecto de elecciones de Senado y Cámaras étnicas, …",positive/neutral,negative


### Analyzing errors TF-IDF

In [43]:
error_df2 = pd.concat(
    [ pd.concat([X_test, y_test ], axis = 1).reset_index(),
    pd.Series(y_test_tfidf_predict) ]
, axis = 1).rename(columns = { 'sentiment': 'actual', 0: 'predicted' })

error_df2.drop('index', inplace = True, axis = 1)

In [44]:
error_df2.shape

(60, 3)

In [45]:
error_df2.loc[error_df2['actual'] != error_df2['predicted']].head(20)

,full_text,actual,predicted
8,"@tinagus2000 Evidentemente la persecución judicial, la aceleración de cosas, las trabas del @cnegobec para la inscripción , el viaje de la fiscal a Colombia y el papel de otras instituciones influyeron en la victoria de Lasso. Al otro candidato lo desgastaron, mucha antes de las elecciones.",positive/neutral,negative
16,"Sin importar la orilla política en la que se esté, es preciso reconocer que Álvaro Uribe ha sido el que les dio forma a los últimos 20 años de la política colombiana. \r\n\r\n#ContenidoPremiumEE #Elecciones2022 \r\nhttps://t.co/tVxPTMLkC2",positive/neutral,negative
18,Petro promete reanudar relaciones diplomáticas con Maduro si gana elecciones en Colombia\r\n#TalCual #ClaroyRaspao #AmigosDeNuevo \r\nhttps://t.co/Kpm7VZydwZ https://t.co/xyy0uPwwzZ,negative,positive/neutral
20,Eso sabe ;)\r\nAdelante #PetroPresidente2022 \r\nhttps://t.co/xscQ26b62m https://t.co/NIJNT0Sms7,positive/neutral,negative
23,Me salió esto 😅\r\n\r\nhttps://t.co/TJokUajStC,positive/neutral,negative
24,"@condeza6 @Gonzalo00333993 @Pura_Miel @Registraduria @CNE_COLOMBIA @moecolombia @PGN_COL Coño que porquería de país , acaso la procuradora se revolcó con el registrador y le tomo fotos y videos que la tiene chantajeada o que , que carajos es lo que esa vieja está esperando que se forme un mierdero el día de las elecciones?",positive/neutral,negative
28,"#Colombia #Petro anuncia que reanudará las relaciones con Venezuela. Colombianos si queréis que el salario mínimo de Colombia baje a los 2 dólares al mes como en Venezuela, por debajo del umbral de pobreza (1,75 dólares/día) vuestro candidato es #Petro.\r\nhttps://t.co/LpgzfWPvN7",negative,positive/neutral
29,"🇨🇴 QUE COMIENCE EL SHOW\r\nCOLOMBIANO (CAMINO A ELECCIONES PRESIDENCIALES EN COLOMBIA).\r\n\r\nA 23 dias para la primera ronda presidencial, se devela el pacto que hay detras.\r\n\r\nhttps://t.co/ugZhjFXZOD",negative,positive/neutral
31,Estos son los tarjetones con los que Colombia ha votado a la presidencia en 28 años:\r\n\r\nhttps://t.co/7jcyCwQuni,positive/neutral,negative
32,#Política #Colombia | Uno de los vehículos portaba en su parabrisas un distintivo que lo identificaba como parte de la Alcaldía de Medellín.\r\nhttps://t.co/LDYchqsr08,positive/neutral,negative
